In [140]:
import functools

In [3]:
import os
import sys
os.environ['SPARK_HOME'] = '/home/gleb/spark'
SPARK_HOME = os.environ.get('SPARK_HOME', None)
sys.path.insert(0, SPARK_HOME+ "/python")
sys.path.insert(0, os.path.join(SPARK_HOME, 'python/lib/py4j-0.10.3-src.zip'))

In [4]:
exec(open('/home/gleb/spark/python/pyspark/shell.py').read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.0.2
      /_/

Using Python version 3.5.2 (default, Jul  2 2016 17:53:06)
SparkSession available as 'spark'.


In [23]:
%%sh

mkdir -p mnist && {
    cd mnist;
    wget -q -nc http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz &&
    wget -q -nc http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz &&
    wget -q -nc http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz &&
    wget -q -nc http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz &&
    gunzip *.gz
}

gzip: t10k-images-idx3-ubyte already exists;	not overwritten
gzip: t10k-labels-idx1-ubyte already exists;	not overwritten
gzip: train-images-idx3-ubyte already exists;	not overwritten
gzip: train-labels-idx1-ubyte already exists;	not overwritten


In [5]:
import os, struct
from array import array as pyarray
from numpy import append, array, int8, uint8, zeros
import numpy as np
def load_mnist(dataset="training", digits=np.arange(10), path="."):
    """
    Loads MNIST files into 3D numpy arrays

    Adapted from: http://abel.ee.ucla.edu/cvxopt/_downloads/mnist.py
    """

    if dataset == "training":
        fname_img = os.path.join(path, 'train-images-idx3-ubyte')
        fname_lbl = os.path.join(path, 'train-labels-idx1-ubyte')
    elif dataset == "testing":
        fname_img = os.path.join(path, 't10k-images-idx3-ubyte')
        fname_lbl = os.path.join(path, 't10k-labels-idx1-ubyte')
    else:
        raise ValueError("dataset must be 'testing' or 'training'")

    flbl = open(fname_lbl, 'rb')
    magic_nr, size = struct.unpack(">II", flbl.read(8))
    lbl = pyarray("b", flbl.read())
    flbl.close()

    fimg = open(fname_img, 'rb')
    magic_nr, size, rows, cols = struct.unpack(">IIII", fimg.read(16))
    img = pyarray("B", fimg.read())
    fimg.close()

    ind = [ k for k in range(size) if lbl[k] in digits ]
    N = len(ind)

    images = zeros((N, rows, cols), dtype=uint8)
    labels = zeros((N, 1), dtype=int8)
    for i in range(len(ind)):
        images[i] = array(img[ ind[i]*rows*cols : (ind[i]+1)*rows*cols ]).reshape((rows, cols))
        labels[i] = lbl[ind[i]]

    return images, labels

In [6]:
X, y = load_mnist(dataset='training', path='mnist/')
X = X.reshape(-1, 1, 28, 28)

X_test, y_test = load_mnist(dataset='testing', path='mnist/')
X_test = X_test.reshape(-1, 1, 28, 28)

In [29]:
X_train_rdd = sc.parallelize(
    (i, y[i], X[i].ravel().copy())
    for i in range(X.shape[0])
).repartition(64).persist()

X_test_rdd = sc.parallelize(
    (i, y_test[i], X_test[i].ravel().copy())
    for i in range(X_test.shape[0])
).repartition(64).persist()

In [225]:
class LogisticRegression(object):
    def __init__(self):
        self.W = np.random.uniform(-1,1, size = (28*28,10)) #10 classes, 28*28 features
        self.b = np.random.uniform(-1,1, size = (10,))
        self.I = np.eye(10,10)
        
    def predict(self,x): 
        dot = np.dot(x,self.W)+self.b
        x_max = np.max(dot)
        exps = np.exp(dot-x_max)
        return exps/np.sum(exps)
    
    def next_iter(self,X,batch_size, learning_rate = 0.9):
        mini_batch = X.sample(withReplacement=False, fraction = batch_size).cache()
        N = 1/(batch_size*60000)
        #index,target,features
        W_grad = mini_batch.map(lambda x: N*np.outer(x[2], self.I[x[1]]-self.predict(x[2]))).reduce(lambda a,b: a+b)
        b_grad = mini_batch.flatMap(lambda x: N*(self.I[x[1]]-self.predict(x[2]))).reduce(lambda a,b: a+b)
        self.W += learning_rate*W_grad
        self.b += learning_rate*b_grad
        
    def get_accuracy(self, X):
        predicted = X.flatMap(lambda x: np.argmax(self.predict(x[2])) == x[1]).collect()
        return sum(predicted)/X.count()
    
    def fit(self, X, batch_size = 0.3, learning_rate=0.9, num_iter=100):
        for i in range(num_iter):
            print(i)
            self.next_iter(X,batch_size,learning_rate)
            #print('Iteration {0} -----> {1: .2f}% accuracy'.format(i,get_accuracy(X_test_rdd)))

In [226]:
l = LogisticRegression()


In [227]:
l.fit(X_train_rdd,num_iter=100)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [229]:
l.get_accuracy(X_test_rdd)

0.84740000000000004

In [212]:
X_test_rdd.count()

10000